# 微调（Fine-Tuning）

---

## 一、微调是什么？

> 别人在大数据集上训练好了一个模型，拿过来，改一改最后一层，在自己的小数据集上接着训练。

**为什么这样做？**

- 你自己的数据通常很少（比如5万张、100类）
- 从零训练一个好模型需要海量数据（ImageNet有120万张）
- 别人训练好的模型已经学会了"看图片"的能力，你只需要在此基础上微调

---

## 二、神经网络的两部分

把任何一个分类网络想成两块：

```
图片 → [特征提取（很多层）] → [分类器（最后一层全连接）] → 预测结果
```

| 部分 | 作用 | 微调时怎么处理 |
|:--|:--|:--|
| 特征提取（前面所有层） | 把像素变成有意义的特征 | **从预训练模型复制过来**（不是随机初始化） |
| 分类器（最后一层FC） | 根据特征判断类别 | **随机初始化**（因为你的类别和ImageNet不同） |

**核心想法：** 特征提取的能力是通用的（边缘、纹理、形状……），换个数据集也能用。但分类器是跟类别绑定的，必须重新来。

---

## 三、微调 vs 从零训练

| | 微调 | 从零训练 |
|:--|:--|:--|
| 特征提取层初始化 | 用预训练权重 | 随机 |
| 最后一层初始化 | 随机（类别数不同） | 随机 |
| 学习率 | 小（已经接近最优了） | 正常大小 |
| 训练轮数 | 少（1~5个epoch） | 多（几十上百个epoch） |
| 最终精度（热狗数据集） | **~94%** | ~84% |

**就改了一个地方：初始化用预训练权重而不是随机**

---

## 四、代码实现

### 第1步：数据准备

```python
# 这里的 normalize 必须和 ImageNet 预训练时用的一样
# 因为预训练模型"习惯"了这种输入分布
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],   # ImageNet 三个通道的均值
    std=[0.229, 0.224, 0.225]     # ImageNet 三个通道的标准差
)
```

> **为什么测试也要 normalize？** 不是为了增广，而是预训练模型训练时输入就是 normalize 过的。你喂原始数据进去，它"看不懂"。

```python
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),     # ImageNet模型要求224×224
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,     # 和预训练保持一致
])

test_transform = transforms.Compose([
    transforms.Resize(256), # 短边缩放到256，长边等比例跟着变
    transforms.CenterCrop(224), # 从中心裁出224×224
    transforms.ToTensor(),
    normalize,
])

train_dataset = datasets.ImageFolder('data/hotdog/train', transform=train_transform)
test_dataset  = datasets.ImageFolder('data/hotdog/test',  transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=64)
```

---

### 第2步：加载预训练模型并修改最后一层

```python
from torchvision import models
import torch.nn as nn

# pretrained=True：不仅下载模型结构，还下载在ImageNet上训练好的权重
pretrained_net = models.resnet18(pretrained=True)
```

> **`pretrained=True` vs `pretrained=False`：**
> - `True`：权重是ImageNet上训练好的（微调用这个）
> - `False`：权重是随机初始化的（从零训练用这个）

```python
# 看一下最后一层长什么样
print(pretrained_net.fc)
# Linear(in_features=512, out_features=1000, bias=True)
# 输入512维特征，输出1000类（ImageNet的1000个类别）
```

```python
# 我们的任务只有2类（热狗 / 不是热狗），所以要替换最后一层
pretrained_net.fc = nn.Linear(512, 2)

# 只对新的最后一层做随机初始化
nn.init.xavier_uniform_(pretrained_net.fc.weight)
```

> **`nn.Linear(512, 2)`：** 创建一个新的全连接层，输入512维，输出2类。这一层的权重是随机的，前面所有层的权重还是ImageNet预训练的。

> **`nn.init.xavier_uniform_()`：** 一种比较好的随机初始化方法，比默认的随机效果好一点。`_` 结尾表示原地修改。

**执行完这两行后，模型的状态：**
```
前面所有层 → ImageNet预训练权重（已经很好了）
最后一层FC → 随机初始化（需要重新学）
```

---

### 第3步：设置不同的学习率

**关键技巧：** 前面的层已经很好了，用小学习率微调；最后一层是随机的，用大学习率快速学。

```python
# 把参数分成两组
params_1x = [p for name, p in pretrained_net.named_parameters()
             if 'fc' not in name]   # 不包含'fc'的 = 前面所有层

optimizer = torch.optim.SGD([
    {'params': params_1x, 'lr': 5e-5},              # 前面的层：小学习率
    {'params': pretrained_net.fc.parameters(), 'lr': 5e-4},  # 最后一层：10倍学习率
], momentum=0.9, weight_decay=0.001)
```

> **语法解释：**
> - `named_parameters()`：返回 `(参数名, 参数值)` 的迭代器
> - `'fc' not in name`：名字里不包含 `'fc'` 的参数 = 除最后一层外的所有参数
> - SGD 的参数列表可以传**字典列表**，每组参数用不同的学习率

---

### 第4步：训练（和之前完全一样）

```python
loss_fn = nn.CrossEntropyLoss()
num_epochs = 5   # 微调只需要很少的epoch

for epoch in range(num_epochs):
    pretrained_net.train()
    for X, y in train_loader:
        output = pretrained_net(X)
        loss = loss_fn(output, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
```

**训练代码和从零训练一模一样，区别全在前面的初始化和学习率设置上。**

---

## 五、效果对比

| | 微调（pretrained=True） | 从零训练（pretrained=False） |
|:--|:--|:--|
| epoch 1 精度 | 已经很高了 | 还在慢慢爬 |
| epoch 5 精度 | **~94%** | ~84% |
| 是否还需要更多epoch | 不需要，1-2个就够了 | 还在涨，但很慢 |

---

## 六、整体流程总结

```
微调三步：
  1. 加载预训练模型        pretrained=True
  2. 替换最后一层          fc = nn.Linear(512, 你的类别数)
  3. 分组设学习率          前面的层用小学习率，最后一层用大学习率

其余（数据加载、训练循环、loss计算）和从零训练完全相同。
```

---

## 七、实际情况

**几乎所有情况下，都应该从微调开始，而不是从零训练。**

- 数据少 → 微调效果远好于从零训练
- 数据多 → 微调至少不会更差，而且收敛更快
- 从零训练大模型是大公司/学术界的事，个人和应用层面用微调就够了

----
# 疑问解答

## 一、先搞懂神经网络到底在干嘛

### 用人来类比

假设你从没见过"热狗"，但你从小到大见过几万种东西。

```
你的大脑已经学会了：
  - 什么是"长条形"
  - 什么是"棕色"
  - 什么是"食物的质感"
  - 什么是"面包的纹理"
```

现在有人给你看一张热狗的图，说"这叫热狗"。

**你只看一次就记住了。** 为什么？

因为你不需要重新学"什么是长条形""什么是棕色"，这些底层能力你早就有了。你只需要学一个新的对应关系：**"长条形 + 棕色 + 面包纹理 → 热狗"**。

---

### 神经网络也是这样

```
一张图片（像素）
    ↓
  第1层：学会识别"边缘""线条"
    ↓
  第2层：学会识别"纹理""花纹"
    ↓
  第3层：学会识别"形状""轮廓"
    ↓
  第4层：学会识别"物体的部件"（轮子、眼睛、面包）
    ↓
  ......（可能十几层、几十层）
    ↓
  最后一层（全连接层）：看到这些特征 → 判断是哪一类
```

**前面所有层 = 你的视觉经验（学会"看"东西）**
**最后一层 = 你的分类决策（看到这些特征，判断它叫什么）**

---

### 什么是"全连接层"？

别被名字吓到，它就是一个**打分器**：

```
前面的层提取出了一组特征，比如512个数字：
  [0.8, 0.1, 0.9, 0.3, ...]   ← 512个数字，描述"这张图长什么样"

全连接层做的事：
  热狗的得分 = 0.8×w1 + 0.1×w2 + 0.9×w3 + ... （512个数加权求和）
  不是热狗的得分 = 0.8×v1 + 0.1×v2 + 0.9×v3 + ...

  哪个得分高，就判断是哪类
```

**就是加权求和，选得分最高的类别。** 所谓"全连接"就是每个输入和每个输出之间都有一个权重。

**ImageNet 的最后一层：** 512个输入 → 1000个输出（1000个类别各一个得分）
**我们替换成：** 512个输入 → 2个输出（热狗 / 不是热狗）

---

## 二、为什么能"偷别人的成果"？

### 这是最关键的问题

**答案：因为"看东西"的能力是通用的。**

ImageNet的模型在120万张图上学了1000种东西，它学到的前面那些层的能力：

```
第1层学到的"边缘检测"      → 看热狗也需要
第2层学到的"纹理识别"      → 看热狗也需要
第3层学到的"形状理解"      → 看热狗也需要
第4层学到的"物体部件识别"   → 看热狗也需要
```

**这些能力不是专门为ImageNet的1000类服务的，是通用的视觉能力。**

唯一不通用的是最后一层——"看到这些特征后判断是1000类中的哪一类"。因为我们的任务只有2类，所以最后一层必须换掉。

### 类比

```
一个学了10年画画的人，转去学摄影：
  - 他对构图、光影、色彩的理解 → 直接能用（通用能力）
  - 他只需要学相机操作 → 很快上手（新任务特定的部分）

一个从没接触过视觉艺术的人，学摄影：
  - 构图、光影、色彩 → 全部从零学
  - 相机操作 → 也要学
  - 慢得多，效果也差
```

**微调就是：把别人的"视觉经验"拿过来，只重新学"分类决策"。**

---

## 三、代码实现（真正该记住的部分）

### 整个微调只比从零训练多了三行关键代码

**从零训练：**
```python
net = models.resnet18(pretrained=False)  # 随机初始化
net.fc = nn.Linear(512, 2)              # 改成2分类
```

**微调：**
```python
net = models.resnet18(pretrained=True)   # ← 关键改动1：加载预训练权重
net.fc = nn.Linear(512, 2)              # ← 关键改动2：替换最后一层（自动变成随机）
nn.init.xavier_uniform_(net.fc.weight)  # ← 关键改动3：用更好的随机初始化
```

**就这三行，精度从84%变成94%。**

---

### 学习率分组（可选但推荐的技巧）

```python
# 前面的层：已经很好了，轻轻调
# 最后一层：随机初始化的，需要大力学

params_pretrained = [p for name, p in net.named_parameters() if 'fc' not in name]
params_new = net.fc.parameters()

optimizer = torch.optim.SGD([
    {'params': params_pretrained, 'lr': 5e-5},   # 前面：小学习率
    {'params': params_new,        'lr': 5e-4},   # 最后一层：10倍学习率
])
```

> 为什么最后一层学习率大？因为它是随机的，离正确答案很远，需要大步走。
> 为什么前面学习率小？因为已经很好了，只需要微微调整，大步走反而会把好的东西破坏掉。

---

### 数据处理（必须和预训练时一致）

```python
# 这组数字是 ImageNet 训练时用的均值和标准差，照抄就行
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),  # ImageNet模型要求224×224输入
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,                          # 必须加，因为预训练模型"习惯"了这种输入
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])
```

---

### 训练代码（和从零训练完全一样）

```python
loss_fn = nn.CrossEntropyLoss()

for epoch in range(5):         # 微调只需要很少的epoch
    net.train()
    for X, y in train_loader:
        output = net(X)
        loss = loss_fn(output, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
```

---

## 四、完整对照

```python
# ========== 微调版本 ==========
net = models.resnet18(pretrained=True)        # 用预训练权重
net.fc = nn.Linear(512, 2)                    # 换最后一层
nn.init.xavier_uniform_(net.fc.weight)

optimizer = torch.optim.SGD([
    {'params': [p for n, p in net.named_parameters() if 'fc' not in n], 'lr': 5e-5},
    {'params': net.fc.parameters(), 'lr': 5e-4},
])
# → 5个epoch，精度94%


# ========== 从零训练版本（对比用） ==========
net = models.resnet18(pretrained=False)       # 随机初始化（唯一区别）
net.fc = nn.Linear(512, 2)

optimizer = torch.optim.SGD(net.parameters(), lr=5e-4)
# → 5个epoch，精度84%
```

---

## 五、自检清单

学完这节，你应该能回答以下问题：

| # | 问题 | 你应该能说出的答案 |
|:--|:--|:--|
| 1 | 微调和从零训练的区别是什么？ | `pretrained=True` 还是 `False`，即初始权重是预训练的还是随机的 |
| 2 | 为什么要换最后一层？ | 因为类别数不同（ImageNet是1000类，我的任务是2类） |
| 3 | 为什么前面的层可以直接拿来用？ | 因为提取视觉特征的能力是通用的，跟具体任务无关 |
| 4 | 为什么微调用小学习率？ | 权重已经接近最优了，大学习率会破坏已有的好权重 |
| 5 | 为什么最后一层学习率要大？ | 最后一层是随机初始化的，需要快速学习 |
| 6 | normalize 为什么要和 ImageNet 一样？ | 预训练模型训练时用了这个 normalize，输入分布要一致 |
| 7 | 微调一般需要多少 epoch？ | 很少，1~5个就够了 |
| 8 | 代码上微调和从零训练差在哪？ | 就差 `pretrained=True` 和学习率分组，训练循环完全一样 |